In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib qt5

In [2]:
import numpy as np
import pickle
import psycopg2 as pg

 (id SERIAL PRIMARY KEY, time TIMESTAMP, position INT, timepoint INT, 
                            channelno INT, areas BYTEA, lengths BYTEA, numobjects BYTEA)

In [11]:
con = None
con = pg.connect(database='exp21bp000', user='postgres', password='postgres')
con.autocommit=True
cur = con.cursor()
cur.execute("SELECT * FROM growth")
data = cur.fetchall()
sorted_data = sorted(data, key=lambda element: element[0])
formatted_data = [ [datapoint[0], datapoint[2], datapoint[3], datapoint[4], 
                sum(pickle.loads(datapoint[5])), sum(pickle.loads(datapoint[6])), len(pickle.loads(datapoint[7]))] 
                for datapoint in sorted_data]
data_numpy = np.asarray(formatted_data)
if con:
    con.close()

In [12]:
len(data)

37800

In [16]:
formatted_data[0]

[1, 1, 0, 0, 461, 93.9711471723989, 3]

In [13]:
data_numpy.shape

(37800, 7)

In [20]:
num_positions = int(np.max(data_numpy[:, 1])) + 1
max_channels = int(np.max(data_numpy[:, 3]))  + 1

In [56]:
area_cutoff = 1000
time_cutoff=13
fraction = 0.5
blobs_cutoff = 10
for pos in range(num_positions):
    for channel in range(max_channels):
        pos_channel_data = data_numpy[np.argwhere(np.logical_and(data_numpy[:, 1] == pos, data_numpy[:, 3] == channel))]
        #print(f"Pos: {pos} -- channelno: {channel} -- data: {len(pos_channel_data)}")
        # sort the data by time and apply filters and select positions
        pos_channel_data_list = pos_channel_data.squeeze(1).tolist()
        channel_data = np.asarray(sorted(pos_channel_data_list, key=lambda element: element[2]))
        
        area_fraction = np.sum(channel_data[time_cutoff:, 4] > area_cutoff)/ channel_data.shape[0]
        blobs_fraction = np.sum(channel_data[time_cutoff:, 6] > blobs_cutoff)/ channel_data.shape[0]
        
        #print(blobs_fraction)
        if area_fraction > fraction and blobs_fraction > fraction:
            print(pos, channel)

0 102
1 95
5 97


In [47]:
sorted_by_time[13:, 4]

array([2960., 2384., 2408., 2374., 2466., 2438., 2471., 2368., 2454.,
       2368., 2430., 1824., 2155., 2226., 2253., 2281., 2263., 2341.,
       2300., 2389., 2412., 2401., 2403., 2319., 2310., 2329., 2301.])

In [48]:
len(sorted_by_time)

40